## Jaskeerat Brar
## Individual Project 1

----

### Craigslist & The Old Reader
#### Part 1

In [ ]:
#1
# Identify the Target:
#Start with navigating to the “free” section on the Craigslist San Francisco Bay Area site (https://sfbay.craigslist.org/search/zipLinks to an external site.).

#This page lists items that people are giving away for free.

In [ ]:
#2
#Interact with the Page-Sorting:
#Initially, the listings might be sorted by “newest” first.  Try changing the sorting order to “oldest” first by interacting with the page’s UI.

#Observe any changes in the URL after you change the sorting order back and forth.


**Can you trigger the sorting change directly by modifying only the URL in your browser’s address bar?  If so, how?**

Yes I can directly modify the url and sort the listings from newest to oldest without any javascript / interaction.
The change would be like:

newest : https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery~0~0

change url to:https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0

simply add 'oldest' behind the date text.

**Explain what type of request is made when you change the sort order (GET or POST).**

This is a GET request. This is because we are **requesting** information from server. Our motive is to search.

**What is the variable in the URL associated with sorting?**

The variable in url which is associated with sorting looks to be: **sort**


***Answer***:



In [ ]:
#3
#Interact with the Page-Pagination:
#Craigslist paginates listings, typically displaying a limited number of items per page (120 for me).

#Navigate to the second and third pages of results and observe the changes in the URL.

#Exploration Task:

#Determine how to move between pages by only changing the URL.  What part of the URL changes as you navigate through different pages?

#This task will help you understand how pagination works on Craigslist and how you can programmatically access different pages of listings.



**Determine how to move between pages by only changing the URL.  What part of the URL changes as you navigate through different pages?**

As we navigate **between** pages, the url undergoes very subtle changes. Please check comparison below:

first page: https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~0~0

next page : https://sfbay.craigslist.org/search/zip?sort=dateoldest#search=1~gallery~1~0

The trailing digits at the end of url seem to dictacte how to move between the pages. In the case of between webpages, the first digit changes from 0 to 1 and then so forth. (~0~0) --> (~1~0)


**Identify the variable associated with page changes.  How does altering this variable in the URL affect the page you’re viewing?**

The variable associated with page changes is : search=1~gallery~1~0

We already found how changing the first digit in this variable (~ **1** ~0) helps us navigate between pages.

upon further dwelling, I found that the second digit signifies the starting index of number of items displayed on the browser.
For example, the max items which can be displayed is 120.

Once you scroll past some items, the digit will change to something like ~1~30

This means, the webpage is now displaying you items listed on the second page which are 30th in the list on that webpage.

***Explain***

In [ ]:
#4
#Fetch Listing URLs:
#Use `requests` to access the first page of the “free” section, ordered “newest” first.

#Deploy `BeautifulSoup` to parse the HTML content.

#Extract the first 250 unique listing URLs and save them to a list.  Consider the pagination feature of Craigslist to navigate through pages.
# Explain your strategy.

#Print the list to screen.

**Identify the structure that holds the links to individual listing pages.  What selector do you choose to grab the link?**

I identified that all the listings are housed under a class list items denoted by 'li'.
So I grab all the list items under this particular class: 'li', class_='cl-static-search-result'
and then perform regex to find the url of listing in this class.

**Can you identify one more possible selection method to retrieve the link to the individual listing?  Explain.**
I could have directly searched for 'a' tags and then extracted href component from the webpage.
This poses a risk that my scraper could identify links which are not listings.

Hence for a robust list / items scraper, I only identify links which are housed under the **li** tags.

**Strategy**

So I have identified that below is the base url I need to work with:

https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery

Above URL remains same for the most part. I need to tinker around with the digits at the end of url to paginate between pages and listings.

Essentially, my code will grab all the listings urls from the first webpage.
Then it realizes that we haven't reached 250 urls.

Once we reach the threshold, I will increase the digit in the url to go to next page, grab url and so forth.


In [1]:
import time

import requests
from bs4 import BeautifulSoup

import math

# I will capture 250 urls in a list, let's initialize it here first

numListings = 250

listingUrls = []

# Cg limits 120 items per page
cgListingsCap = 120

# Define agent (In our case it's the famous James Bond), 
# we want craigslist server to think it's a human performing these actions

jamesBond = {'User-Agent': 'Mozilla/5.0'}


In [124]:
## here I define the logic
## --> How many times do I need to query Craiglist to get 250 items? You can change the variable numListings to whatever
## number you like, my logic will calculate the number of requests needed to fetch that many listings.

#numRequestsNeeded = math.ceil(numListings / cgListingsCap)


#print(numRequestsNeeded)

In [126]:
## This for loop will essentially fetch you the number of listings desired.
## in our case, it will get you 250 listings

# Base URL
baseUrl = "https://sfbay.craigslist.org/search/zip?sort=date#search=1~gallery"

listingUrls = []

num = 0

#startIndex = num * 120

while len(listingUrls) < (numListings):
    
    page_url = f"{baseUrl}~{num * 120}"
    
    time.sleep(2)
    
    response = requests.get(page_url, headers=jamesBond)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    list_items = soup.find_all('li', class_='cl-static-search-result')
    
    for item in list_items:
        
        a_tag = item.find('a')
        
        if a_tag and 'href' in a_tag.attrs:
            listingUrls.append(a_tag['href'])
    
    num+=1
    
    if listingUrls == 250:
        break
    

In [127]:
len(listingUrls)

360

In [128]:
# Trim the list to first 250 items
listingUrls = listingUrls[:250]

In [129]:
print(listingUrls)

['https://sfbay.craigslist.org/eby/zip/d/hayward-craftsman-ratchet-needs-fixing/7715602539.html', 'https://sfbay.craigslist.org/eby/zip/d/hayward-lan-cable/7711635226.html', 'https://sfbay.craigslist.org/eby/zip/d/san-ramon-52-sony-tv-with-mount/7715598488.html', 'https://sfbay.craigslist.org/eby/zip/d/union-city-1000-piece-sided-puzzle/7714897627.html', 'https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-phantom-dimmable/7715591270.html', 'https://sfbay.craigslist.org/eby/zip/d/oakland-very-comfortable-vintage-chair/7708960154.html', 'https://sfbay.craigslist.org/eby/zip/d/oakland-comfy-sofa/7715586833.html', 'https://sfbay.craigslist.org/sby/zip/d/cupertino-recliner/7709238646.html', 'https://sfbay.craigslist.org/pen/zip/d/mountain-view-heavy-duty-mop/7708510254.html', 'https://sfbay.craigslist.org/pen/zip/d/mountain-view-six-dinning-chairs/7715586035.html', 'https://sfbay.craigslist.org/pen/zip/d/atherton-free-pool-stuff/7715585874.html', 'https://sfbay.craigslist.org/eby/zip

----

In [ ]:
#5
#Save HTML Pages:
#For each of the 250 listing URLs, use `requests` to fetch the listing page.

#Save each HTML content to a separate file on disk.  Use each listing’s ID to organize files in a way that makes
#them easily identifiable (e.g., save listing ID 7713901653 to file “7713901653.html”).

In [ ]:
## Now we need to query each link, save its response as an html file to our drive


In [130]:
import re
import os


saveDirectory = "freeListingsDirectory"

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)
    
iteration = 0
    

for url in listingUrls:
    
    ## grab the trailing number from url to form the name of to be saved file
    
    parts = url.split('/')
    last_part = parts[-1] if parts else ''
    number_part = last_part.split('.')[0]
    
    if number_part:
        
        fileName = f"{number_part}.html"
        
        time.sleep(5)
        
        ## Uncomment below line to perform requests. I didn't want to mistakenly run it again and
        ## have it over write my files
        
        ###########################
                ###########################
                    ###########################
        
        #response = requests.get(url, headers = jamesBond)
        
                ###########################
                    ###########################
                        ###########################
        
        if response.ok:
        
            with open(os.path.join(saveDirectory, fileName), 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"Saved {url} as {fileName}")
            
        else:
            print(f"Response failed for : {url}")
            
    print(iteration)
    iteration += 1

Saved https://sfbay.craigslist.org/eby/zip/d/hayward-craftsman-ratchet-needs-fixing/7715602539.html as 7715602539.html
0
Saved https://sfbay.craigslist.org/eby/zip/d/hayward-lan-cable/7711635226.html as 7711635226.html
1
Saved https://sfbay.craigslist.org/eby/zip/d/san-ramon-52-sony-tv-with-mount/7715598488.html as 7715598488.html
2
Saved https://sfbay.craigslist.org/eby/zip/d/union-city-1000-piece-sided-puzzle/7714897627.html as 7714897627.html
3
Saved https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-phantom-dimmable/7715591270.html as 7715591270.html
4
Saved https://sfbay.craigslist.org/eby/zip/d/oakland-very-comfortable-vintage-chair/7708960154.html as 7708960154.html
5
Saved https://sfbay.craigslist.org/eby/zip/d/oakland-comfy-sofa/7715586833.html as 7715586833.html
6
Saved https://sfbay.craigslist.org/sby/zip/d/cupertino-recliner/7709238646.html as 7709238646.html
7
Saved https://sfbay.craigslist.org/pen/zip/d/mountain-view-heavy-duty-mop/7708510254.html as 7708510254.htm

Saved https://sfbay.craigslist.org/eby/zip/d/oakland-free-cd-dvd-organizer-sheets/7715559010.html as 7715559010.html
72
Saved https://sfbay.craigslist.org/sfc/zip/d/san-francisco-pick-up-pending-free-tv-32/7715558608.html as 7715558608.html
73
Saved https://sfbay.craigslist.org/eby/zip/d/concord-tesla-19-inch-wheelcovers-fit/7715557666.html as 7715557666.html
74
Saved https://sfbay.craigslist.org/eby/zip/d/rodeo-free-must-go-boxes-padding/7715557648.html as 7715557648.html
75
Saved https://sfbay.craigslist.org/eby/zip/d/alameda-free-furniture-tv-stand-mirrors/7711517676.html as 7711517676.html
76
Saved https://sfbay.craigslist.org/eby/zip/d/concord-ge-electric-range/7715556669.html as 7715556669.html
77
Saved https://sfbay.craigslist.org/pen/zip/d/portola-valley-free-large-leather/7715555821.html as 7715555821.html
78
Saved https://sfbay.craigslist.org/sfc/zip/d/san-francisco-sleeper-sofa-in-neutral/7708264817.html as 7708264817.html
79
Saved https://sfbay.craigslist.org/eby/zip/d/alam

Saved https://sfbay.craigslist.org/sby/zip/d/san-jose-free-ps4-with-controllers-no/7715517244.html as 7715517244.html
143
Saved https://sfbay.craigslist.org/eby/zip/d/hayward-artificial-christmas-tree/7715516846.html as 7715516846.html
144
Saved https://sfbay.craigslist.org/sby/zip/d/san-jose-solid-wood-dinning-room-set/7714234269.html as 7714234269.html
145
Saved https://sfbay.craigslist.org/scz/zip/d/aptos-free-wooden-rounds/7715514935.html as 7715514935.html
146
Saved https://sfbay.craigslist.org/eby/zip/d/alameda-task-chair-in-floral-print/7715514006.html as 7715514006.html
147
Saved https://sfbay.craigslist.org/eby/zip/d/lafayette-treadmill/7715509943.html as 7715509943.html
148
Saved https://sfbay.craigslist.org/sby/zip/d/san-jose-bath-light-bar-light/7715509713.html as 7715509713.html
149
Saved https://sfbay.craigslist.org/nby/zip/d/sebastopol-vinyl-floor-tiles/7715509520.html as 7715509520.html
150
Saved https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free-binders/77155086

Saved https://sfbay.craigslist.org/eby/zip/d/martinez-free-storage-cabinet-plus-ill/7715449942.html as 7715449942.html
214
Saved https://sfbay.craigslist.org/eby/zip/d/lafayette-free-desk-chair-with-arms/7715449570.html as 7715449570.html
215
Saved https://sfbay.craigslist.org/sfc/zip/d/san-francisco-free/7714735168.html as 7714735168.html
216
Saved https://sfbay.craigslist.org/sfc/zip/d/san-francisco-construction-safety/7714717921.html as 7714717921.html
217
Saved https://sfbay.craigslist.org/pen/zip/d/menlo-park-free-fig-tree/7715447828.html as 7715447828.html
218
Saved https://sfbay.craigslist.org/eby/zip/d/martinez-free-mirror/7715447784.html as 7715447784.html
219
Saved https://sfbay.craigslist.org/nby/zip/d/windsor-grape-gondola/7715447740.html as 7715447740.html
220
Saved https://sfbay.craigslist.org/eby/zip/d/lafayette-free-desk-chair/7715447619.html as 7715447619.html
221
Saved https://sfbay.craigslist.org/sfc/zip/d/free-bags-of-sand-construction/7712106559.html as 7712106559.

----

### Part 2: Parsing and Displaying Information from Saved HTML

In [ ]:
#1 # Read Saved HTML Files

#2 #
#Extract Information:
#For each HTML file, use `BeautifulSoup` to parse the file content.

#Extract and print the following details:

#Title: The title of the listing.

#URL of first image (if an image exists):  The URL of the displayed image.
#It can be found in the `src` attribute of `<img>`

#Description: The full description text of the listing.

#Post ID: Usually found at the bottom of the page or within the page's HTML structure.

#Posted Date: The date when the listing was originally posted.

#Last Updated Date: The date when the listing was last updated.

In [131]:


# Loop through each file in the directory

directory = "freeListingsDirectory"

#itles = []

#mgSrcLinks = []

#escriptionList= []

postingTime = ""
updatedTime = ""


for filename in os.listdir(directory):

    # Check if the file ends with .html

    if filename.endswith(".html"):

        # Construct the full file path

        filepath = os.path.join(directory, filename)

        # Read file to string

        with open(filepath, 'r', encoding='utf-8') as file:

            html = file.read()
            
            soup = BeautifulSoup(html, 'html.parser')
            
            #Tag Finding
            
            title_tag = soup.find('title')
            img_tag = soup.find('img')
            description_tag = soup.find('section', id='postingbody')
            
            ## I will extract post id, posted time and updated time in one single scoop
            ## they all are in one single div class called postinginfos
            final_tags = soup.find('div', class_='postinginfos')
            
            
            ##############################################################
            if title_tag:
                
                full_title = title_tag.get_text()
                
                # slit by - and take the first part
                title = full_title.split(" - ")[0]
                
                titles.append(title)
            ##############################################################
            
            if img_tag and 'src' in img_tag.attrs:
                
                img_src = img_tag['src']
                
                #mgSrcLinks.append(img_src)
                
            else: img_src = "No Image Found"
                
            ##############################################################
                
            
            if description_tag:
                descText = description_tag.get_text(separator=' ', strip=True)
               #descriptionList.append(descText)
            else: descText = "No Description Found"
                
            ##############################################################
                
            if final_tags:
                
                post_id_tag = final_tags.find('p', class_='postinginfo')

                # find post id
                if post_id_tag:
                    post_id = post_id_tag.get_text(strip=True).split(': ')[1]
                    
            ##############################################################
            
            if final_tags:
                
                postTimings = soup.find_all('p', class_= 'postinginfo reveal')
                
                for x in postTimings:
                    if 'posted' in x.text:
                        postingTime = x.find('time', class_='date timeago').text.strip()
                    if 'updated' in x.text:
                        updatedTime = x.find('time', class_='date timeago').text.strip()
                    else: updatedTime = "Not Found"
                
                
            ##############################################################            
                
            
            print(f"Title: {title}")
            print(f"URL of First Image: {img_src}")
            print(f"Description: {descText}")
            print(f"Post Id: {post_id}")
            print(f"Posted Date: {postingTime}")
            print(f"Last Updated Date: {updatedTime}")
            print("\n")         
            


Title: 3 drawer lateral file cabinet
URL of First Image: https://images.craigslist.org/00Q0Q_5aDc88w6ZRt_0t20CI_600x450.jpg
Description: QR Code Link to This Post Giving away a lateral file cabinet in excellent condition. I dont have the keys, so it wont lock. other than that it works great. I am having a bunch of stuff hauled away on Saturday morning, so you need to pick it up before the haulers get there. Please dont ask if it is available, and don't tell me you want it if you are not going to show up.
Post Id: 7715526266
Posted Date: 2024-02-07 16:39
Last Updated Date: Not Found


Title: FREE TEDDY BEARS & GALAXY ROSES FOR VALENTINE'S DAY TO SPREAD LOVE
URL of First Image: https://images.craigslist.org/00Y0Y_8bQG1tUwXSq_0cI0oc_600x450.jpg
Description: QR Code Link to This Post FREE TEDDY BEARS, VALENTINE PLUSH TOYS & GALAXY ROSES FOR VALENTINE'S DAY TO SPREAD LOVE on GARAGE SALE FRIDAY FEB 9, SATURDAY FEB 10 AT AND SUNDAY FEB 11 AT 249 SAGAMORE STREET IN SF FROM 10AM TO 1PM Happy Va

Title: solid wood dinning room set
URL of First Image: https://images.craigslist.org/00Q0Q_50ij0RLy3Nc_0CI0t2_600x450.jpg
Description: QR Code Link to This Post This set comes with dinning table, one leaf, and six chairs. The wood and and upholstery are in good condition. The chairs are on wheels, they also spin and lean back. you pick up
Post Id: 7714234269
Posted Date: 2024-02-03 18:29
Last Updated Date: 2024-02-07 15:59


Title: Books
URL of First Image: https://images.craigslist.org/00V0V_kqLtMCVBU7y_0ut0t2_600x450.jpg
Description: QR Code Link to This Post Dave Ramsey: The Total Money Makeover Suze Orman: The Money Book for the Young, Fabulous & Broke Wolf Hall: Book One of the Thomas Cromwell Trilogy Intelligence Reframed The Epidemic: The Rot of American Culture, Absentee and Permissive Parenting and the Resultant Plague of Joyless, Selfish Children Please take all books and regift any Please lmk when you can pick up (near el cerrito plaza Bart), thanks!
Post Id: 7715542855
Post

Title: IKEA Sparsam bulbs. Brand new
URL of First Image: https://images.craigslist.org/00r0r_2nHG4bW1aQ8_0MM132_600x450.jpg
Description: QR Code Link to This Post Brand new in package.  2 bulbs This is SPARSAM! Message with time for porch pick up.
Post Id: 7715416308
Posted Date: 2024-02-07 11:29
Last Updated Date: Not Found


Title: Desk free
URL of First Image: https://images.craigslist.org/00E0E_754d5rm5KGJ_0t20CI_600x450.jpg
Description: QR Code Link to This Post Desk free On the curb 5744 canna ln San Jose ca 95124
Post Id: 7715582939
Posted Date: 2024-02-07 22:42
Last Updated Date: Not Found


Title: Free floating shelf
URL of First Image: https://images.craigslist.org/00f0f_bAHOWTz6CjU_0t20CI_600x450.jpg
Description: QR Code Link to This Post About 40 inches long, includes hardware.
Post Id: 7715562794
Posted Date: 2024-02-07 19:36
Last Updated Date: Not Found


Title: Smart cat touchpad
URL of First Image: https://images.craigslist.org/00r0r_981rQ2BDVyY_0lM0t2_600x450.jpg
Descr

Title: Dining chairs
URL of First Image: https://images.craigslist.org/00b0b_7zAl3WUSuRg_0CI0t2_600x450.jpg
Description: QR Code Link to This Post 6 Dining chairs Dark brown “leather” dining chairs from Studio D (design shop off Fillmore from several years old). Fair condition, a couple of legs need to be tightened but otherwise fine 38” tall at back 19” tall at seat 15” wide at front of seat Pick up near Geary and 21st
Post Id: 7715540749
Posted Date: 2024-02-07 17:35
Last Updated Date: Not Found


Title: Free electronics
URL of First Image: https://images.craigslist.org/00S0S_6eZ7wSP7vdq_0CI0t2_600x450.jpg
Description: QR Code Link to This Post See pics for free items available. 1. Netgear Cable Modem, model CM400. It is designed to meet DOCSIS 3.0 specifications and is backward compatible with DOCSIS 2.0 networks. 2. Orbi Satellite RBS10. The Netgear Orbi Mesh WiFi Satellite is a WiFi add-on satellite that can extend your existing Orbi WiFi System. It can increase coverage up to 1,5

Title: Craftsman Ratchet needs fixing
URL of First Image: https://images.craigslist.org/01212_aRyJzlO4wuK_0t20CI_600x450.jpg
Description: QR Code Link to This Post Maybe you can fix this. It only ratchets well in one direction. Looks like there is a snap ring to disassemble it. 1/2” drive, made in USA
Post Id: 7715602539
Posted Date: 2024-02-08 04:06
Last Updated Date: Not Found


Title: Chaise lounge
URL of First Image: https://images.craigslist.org/00A0A_dLKYFSVNzAh_0t20CI_600x450.jpg
Description: QR Code Link to This Post Very simple but works! Leave name and number for address if interested.
Post Id: 7715546331
Posted Date: 2024-02-07 18:01
Last Updated Date: Not Found


Title: FREE
URL of First Image: https://images.craigslist.org/00000_kzrf2KyEjbj_0t20CI_600x450.jpg
Description: QR Code Link to This Post DO NOT ASK ME IF AVAILABLE! Boxes of various sizes and padding. There is a bunch. Ideally, must be able to take it all! MUST GO! First come, first served.
Post Id: 7715557648
Pos

------------

### Part 3: Automating Login on The Old Reader

#1.
Creating and Verifying a The Old Reader Account
Account Creation:  Create an account on https://theoldreader.comLinks to an external site..  Use an email address and password that you are comfortable sharing with us.

Manual Login Verification: Before automating the login process, ensure you can manually log in to theoldreader.com with your new credentials.  This confirms that your account is active and your credentials are correct.

Email : jatt123wow@gmail.com
password: bondJames


------

#2. Exploring the Login Mechanism
Navigate to the login page of https://theoldreader.comLinks to an external site..

Use your browser’s developer tools to inspect the page, focusing on the <form> tag involved in the login process.

Document all `<input>` fields within the login form, paying special attention to their name attributes. These fields are crucial for submitting the login request programmatically.

**I've recognized the input fields and have documented them below**

---

#3. 
Analyzing Network Traffic for Login Request
With the network tab of your browser’s developer tools open, log in to the site again.

Identify the network request made when you submit the login form (GET or POST).  Explain why this method was chosen.

Carefully examine the payload that was submitted to the server during login.  Compare this payload to the `<form>` / `<input>` fields you previously analyzed.  Explain your observation.

***Explanation***

The network request made in this case was a POST request.
In our case, POST method is sending a payload / information to the server, who then will use it to verify / cross match our login credentials.

Our requests motive is to deliver a payload. server verifies payload and then let's us authenticate ourselves.


***Examining the payload***

Here's the payload which we dumped to the website

utf8=%E2%9C%93&authenticity_token=hbAsdI%2BHIW2zfuTgPAhFXzeMQRSC%2FIHtTdWlC4%2FwxQU%3D&user%5Blogin%5D=jatt123wow%40gmail.com&user%5Bpassword%5D=bondJames&commit=Sign+In

The Payload is well constructed,

The first part is utf8=%E2%9C%93, which denotes the payload has utf 8 encoding followed by a check mark  ✓ 

Then follows the authenticity token, followed by my login ID/ email, then password

and then finally commit=Sign+In. THis is signalling to server that a post request is coming in towards the Sign In procedure.


In **comparison** to the form/ input fields, the data I checked earlier was simply denoting the formatting of form.
It was also defined across multiple lines.

Payload is one single statement which contains a lot of information necessary to facilitate my login.





----

#4. 
Automating the Login Process
Using Python and appropriate libraries like requests, simulate the login process.

Create a session object to maintain your login state across multiple requests.

Prepare a payload with your login credentials and other necessary form data identified from the login page and the network analysis.

Send a POST request to the login form’s action URL to log in, using the session object.

In [2]:
## code referenced from professor Jorn's file - neopets.ipynb
time.sleep(5) # 5s

readerUrl = 'https://theoldreader.com/users/sign_in'


page = requests.get(readerUrl, headers = jamesBond)

soup = BeautifulSoup(page.content, 'html.parser')

print(soup)

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
<link href="https://fonts.googleapis.com/css?family=Montserrat:400,600" rel="stylesheet" type="text/css"/>
<!-- Latest compiled and minified JavaScript -->
<script src="https://code.jquery.com/jquery.js"></script>
<script src="//netdna.bootstrapcdn.com/bootstrap/3.0.1/js/bootstrap.min.js"></script>
<link href="https://fonts.googleapis.com/css?family=Source+Code+Pro" rel="stylesheet" type="text/css"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,800" rel="stylesheet"/>
<link href="//s.theoldreader.com/assets/reader/public-c7869a909c7b119a27fb646003828344.css" media="screen" rel="stylesheet" type="text/css">
<link href="//s.theoldreader.com/assets/apple-touch-icon-57x57-86fe1176

In [3]:
time.sleep(5) # 5s

inputAuth = soup.select_one('#new_user input[name=authenticity_token]')

_ref_ck = inputAuth.get('value')

print(_ref_ck)




fCsoR/wyzmX/IfXdWzTFYdOpM1/nK6KHu4MVsQTpnCA=


In [190]:

#authToken = soup.select_one('input[name="authenticity_token"]').get('value')

#username = 'jatt123wow@gmail.com'

#password = 'jamesBond'

In [191]:
#print(authToken)

In [4]:
# Always pause between two requests.
time.sleep(5) # 5s

# An open session carries the cookies and allows you to make post requests
session_New = requests.session()

## constructing payload
res = session_New.post('https://theoldreader.com/users/sign_in', 
                        data = {
                            
                                'authenticity_token': _ref_ck,
                                'user[login]': 'jasbrar@ucdavis.edu',
                                'user[password]': 'jasBrar',
                        }, 
                        headers= jamesBond, timeout = 20)

# This will get us the Cookies.
cookies = session_New.cookies.get_dict()

print(cookies)

{'remember_user_token': 'BAhbB1sGVToaTW9wZWQ6OkJTT046Ok9iamVjdElkIhHQF51uROSnp4e6k%2BhJIiIkMmEkMDUkRmZlZWVmM1NBdGgxR1dERHlBM3ovLgY6BkVU--fe0c0f9973dbca90f4cbd325a1927037d8507acc', 'i_know_you': 'Jas+Brar', '_new_reader_session': 'BAh7CkkiD3Nlc3Npb25faWQGOgZFVEkiJTNmNjZmYmU0NjRlZTEyMjI0NWM0ZTMyYzQ0YWUwNWM1BjsAVEkiGXdhcmRlbi51c2VyLnVzZXIua2V5BjsAVFsHWwZVOhpNb3BlZDo6QlNPTjo6T2JqZWN0SWQiEdAXnW5E5Kenh7qT6EkiIiQyYSQwNSRGZmVlZWYzU0F0aDFHV0REeUEzei8uBjsAVEkiDWxhbmd1YWdlBjsARjoHZW5JIhByZWRpcmVjdF90bwY7AEZJIgYvBjsARkkiEF9jc3JmX3Rva2VuBjsARkkiMVJqQmtSVFVuYTJ1Q2w3eC8yTVdNRUFpT2c1MFdpQmxWRHBQL0VPUmlqUlU9BjsARg%3D%3D--a1bffe8f46854a19904a652682707157c8294256', 'signed_at': '1707420148'}


--------

Verifying Successful Login
After attempting to log in, inspect the cookies saved in the session object to understand the information The Old Reader stores on your computer.

Use the session object to access https://theoldreader.comLinks to an external site..

Verify successful login by checking for the presence of your user information that is only available when logged in.

In [5]:
# Always pause between two requests.
time.sleep(5) # 5s

# This is the easiest way to remain in-session.

page_Two = session_New.get('https://theoldreader.com/', cookies = cookies) 

soup_Two = BeautifulSoup(page_Two.content, 'html.parser')

print(soup_Two.prettify());
# print();


<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
  <link href="//s.theoldreader.com/assets/application-befb06d5a14978388154b51422cef437.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-57x57-86fe1176e14af4907a6fecfe5ca7e3f1.png" rel="apple-touch-icon-precomposed" sizes="57x57"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-114x114-bae89acc41c93261dd962ea6ade08d22.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-72x72-f248503edfa3676f8d58af531aff7e88.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
  <link href="//s.theoldreader.com/assets/apple-touch-icon-144x144-510415291cae9b46a9ca4ac398

In [7]:
import re

elements = soup_Two.find_all(string=re.compile(r'Jas Brar')) 

for element in elements:
    print(str(element.parent))

<script type="text/javascript">
  if (typeof Bugsnag !== "undefined" && Bugsnag !== null) {
    Bugsnag.releaseStage = "production";
    Bugsnag.notifyReleaseStages = ['production'];
    Bugsnag.user = {
      id: "d0179d6e44e4a7a787ba93e8",
      name: "Jas Brar",
      email: "jasbrar@ucdavis.edu"
    }
  }
</script>
<script type="text/javascript">
  var FEEDS = {
    feeds: [{"position":null,"id":null,"subscriptions":[],"title":null,"destroyed":null},{"position":1,"id":"65c52990c70bc2b3b100025b","subscriptions":[{"feed_id":"aed846f0d597789927a7ce68","folder_id":"65c52990c70bc2b3b100025b","position":1,"id":"65c52990c70bc2b3b100025f","icon":"/system/uploads/feed/picture/aed8/46f0/d597/7899/27a7/icon_ce68.ico","url":"https://www.apartmenttherapy.com/main.rss","share_count":null,"star_count":null,"comment_count":null,"sendto_count":null,"title_click_count":null,"body_link_click_count":null,"mark_all_as_read_count":null,"likes_count":null,"title":"Apartment Therapy| Saving the world, one

In [ ]:
### Above Verifies that I'm able to login sucessfully using our automated methods in beautiful soup